# Setting up parameters

Make the simulations perfectly reproducible

In [2]:
psom_set_rand_seed(0);

Simulation parameters:
 * `type_sim`: either `'random'` or `'structured'` connection changes
 * `fdr`: FDR threshold
 * `pi1`: Proportion of true effects in the target network
 * `theta`: The effect size
 * `samps`: Number of simulation samples
 * `nb_roi`: Number of regions
 * `nb_net`: Number of networks
 * `size_net`: Number of regions for each network

In [37]:
type_sim = 'structured';
list_q = [0.01 0.05 0.1 0.2]; % The FDR thresholds
list_pi1 = [0.05 0.1 0.5]; % Number of true positive per family
list_theta = [2 3 5]; % The effect size
nb_samps = 100; % Numer of samples
nb_roi = 210;
nb_net = 12;
size_net = [17 17 17 17 17 17 17 17 17 17 17 23];

# Simulations under the null

Get an estimate of excepted volume of discoveries under the null

In [42]:
perc_null = zeros(length(list_q),1);
for qq = 1:length(list_q)
    niak_progress(qq,length(list_q));
    q = list_q(qq);
    for ss = 1:nb_samps
        z_null = randn(nb_roi*(nb_roi-1)/2,1);
        pce_null = normcdf(z_null);    
        [fdr_null,test_null] = niak_fdr(pce_null,'BH',q);        
        perc_null(qq) = perc_null(qq) + any(test_null);
    end
end
perc_null = perc_null / nb_samps; 

    Percentage done: 50 75 100


In [43]:
perc_null

perc_null =

   0.00000
   0.03000
   0.16000
   0.19000



In [44]:
part = [];
for nn = 1:nb_net
  part = [part nn*ones([1 size_net(nn)])];
end
mat = niak_part2mat(part);
mat_v = niak_mat2vec(mat);

In [52]:
%% Loop over parameters
sens = zeros(length(list_theta),length(list_pi1),length(list_q));
spec = zeros(length(list_theta),length(list_pi1),length(list_q));
fdr = zeros(length(list_theta),length(list_pi1),length(list_q));

for tt = 1:length(list_theta)
  for pp = 1:length(list_pi1)        
    theta = list_theta(tt);
    pi1 = list_pi1(pp);
    fprintf('Simulation perc of true diff %1.3f perc, effect size %1.f. \n',pi1,theta);
    for ss = 1:nb_samps
        
        % Define the target connections 
        mask_true = false(size(mat_v));
        if strcmp(type_sim,'random')
          mask_true(1:ceil(pi1*length(mask_true))) = true;
          mask_true = mask_true(randperm(length(mask_true)));
        elseif strcmp(type_sim,'structured')
          target = ceil(nb_net*rand(1));
          ind = find(mat_v==target);
          ind = ind(randperm(length(ind)));
          ind = ind(1:ceil(pi1*length(ind)));
          mask_true(ind) = true;
        else 
          error('%s is an unknown simulation type',type)
        end
        
        % Simulate p values
        z = randn(nb_roi*(nb_roi-1)/2,1);
        z(mask_true) = z(mask_true) + theta;
        pce = 1-normcdf(z);    
        
        for qq = 1:length(list_q)
          q = list_q(qq);
          % run the test 
          [~,test] = niak_fdr(pce,'BH',q);
          sens(tt,pp,qq) = sens(tt,pp,qq) + sum(test&mask_true)/sum(mask_true);
          spec(tt,pp,qq) = spec(tt,pp,qq) + 1 - sum(test&~mask_true)/sum(~mask_true);
          if any(test)
            fdr(tt,pp,qq) = fdr(tt,pp,qq) + sum(test&~mask_true)/sum(test);
          end
        end
    end
  end
end
sens = sens/nb_samps;
spec = spec/nb_samps;
fdr = fdr/nb_samps;

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.500 perc, effect size 5. 
Simulation perc of true diff 0.050 perc, effect size 2. 
Simulation perc of true diff 0.100 perc, effect size 2. 
Simulation perc of true diff 0.500 perc, effect size 2. 
Simulation perc of true diff 0.050 perc, effect size 3. 
Simulation perc of true diff 0.100 perc, effect size 3. 
Simulation perc of true diff 0.500 perc, effect size 3. 
Simulation perc of true diff 0.050 perc, effect size 5. 
Simulation perc of true diff 0.100 perc, effect size 5. 
Simulation perc of true diff 0.

In [55]:
sens
spec
fdr

sens =

ans(:,:,1) =

   0.0014286   0.0025275   0.0017277
   0.0413187   0.0245055   0.0486649
   0.6952747   0.7332967   0.8249722

ans(:,:,2) =

   0.0071429   0.0043407   0.0052304
   0.0802198   0.0787363   0.1424294
   0.8197802   0.8552198   0.9178509

ans(:,:,3) =

   0.0100000   0.0097802   0.0100567
   0.1092308   0.1157692   0.2114625
   0.8721978   0.8867033   0.9466674

ans(:,:,4) =

   0.015165   0.015220   0.018968
   0.175934   0.174560   0.310364
   0.899341   0.920824   0.968520

spec =

ans(:,:,1) =

   1.00000   1.00000   1.00000
   1.00000   1.00000   1.00000
   1.00000   0.99999   0.99997

ans(:,:,2) =

   1.00000   1.00000   1.00000
   0.99999   0.99999   0.99997
   0.99998   0.99996   0.99982

ans(:,:,3) =

   0.99999   0.99999   0.99999
   0.99999   0.99999   0.99992
   0.99996   0.99992   0.99962

ans(:,:,4) =

   0.99998   0.99998   0.99996
   0.99996   0.99996   0.99973
   0.99991   0.99983   0.99914

fdr =

ans(:,:,1) =

   0.030000   0.030000   0.000000
  